# 🦌 ELK Transformer 🤖

A transformer object that will convert some input source into valid Elk Json. This
example using a transformer around a networkx graph.

In [ ]:
import json
import pathlib

import ipywidgets
import networkx
import traitlets
from IPython.display import display

import ipyelk
import ipyelk.nx

# import ipyelk.tools

In [ ]:
graph = networkx.readwrite.json_graph.node_link_graph(
    json.loads(pathlib.Path("flat_graph.json").read_text(encoding="utf-8"))
)

graph

In [ ]:
dg = ipyelk.nx.Diagram(
    source=ipyelk.nx.NXSource(graph=graph),
)
dg

In [ ]:
dg.refresh()

In [ ]:
dg.pipe.value.value.dict()

In [ ]:
dg.pipe.pipes[0].value.value.dict()

In [ ]:
from typing import Dict, Hashable, Iterator

import networkx as nx

from ipyelk.elements import Edge, HierarchicalElement, Label, Node, Port, Registry
from ipyelk.elements.serialization import build_edge, build_shape_map, iter_elements
from ipyelk.schema.validator import validate_elk_json


def from_nx_node(n: Hashable, d: Dict) -> Node:
    if isinstance(n, Node):
        el = n
    else:
        el = Node(**d)
        if el.id is None:
            el.id = str(n)
    return el


def iter_nx_sources(g: nx.DiGraph) -> Iterator[Hashable]:
    for node, in_degree in g.in_degree():
        if in_degree == 0:
            yield node


def single_root(g) -> bool:
    if len(g) == 0:
        return False
    return nx.is_tree(hierarchy)


def process_hierarchy(graph, hierarchy) -> nx.DiGraph:
    # copy graph to avoid mutation
    hierarchy = hierarchy.copy(as_view=False)

    if not single_root(hierarchy):
        # add new root and connect old roots to new root
        root = Node()
        hierarchy.add_node(root)

        # connect old roots to new root
        for n in iter_nx_sources(hierarchy):
            if n != root:
                hierarchy.add_edge(root, n)

    # check graph and add direction parentage to root if needed
    for n in graph.nodes():
        if n not in hierarchy:
            hierarchy.add_edge(root, n)

    return hierarchy


def get_root(hierarchy):
    for root in iter_nx_sources(hierarchy):
        return root


def as_in_hierarchy(node: HierarchicalElement, hierarchy, el_map):
    # TODO need to handle if given a port or node
    if isinstance(node, Port):
        node = node.parent

    if node in hierarchy:
        return node

    #     if isinstance(node, Node):
    #         node = node.get_id()
    #         if node in hierarchy:
    #             return node
    #     if not isinstance(node, HierarchicalElement):
    #         # should be an identifer to something in the element map
    #         node = el_map[node]

    if isinstance(node, HierarchicalElement):
        node = node.get_id()
    else:
        # should be an identifer to something in the element map
        node = el_map[node]
        if isinstance(node, Port):
            node = node.parent
    assert node in hierarchy, "node not in hierarchy"
    return node


def lca(
    hierarchy: nx.DiGraph,
    node1: HierarchicalElement,
    node2: HierarchicalElement,
    el_map: Dict[str, HierarchicalElement],
) -> HierarchicalElement:
    node1 = as_in_hierarchy(node1, hierarchy, el_map)
    node2 = as_in_hierarchy(node2, hierarchy, el_map)

    ancestor = nx.lowest_common_ancestor(hierarchy, node1, node2)
    if not isinstance(ancestor, HierarchicalElement):
        ancestor = el_map[ancestor]
    return ancestor


def get_owner(edge: Edge, hierarchy, el_map) -> HierarchicalElement:
    u = edge.source
    v = edge.target
    return lca(hierarchy, u, v, el_map)

In [ ]:
hierarchy = None
if hierarchy is None:
    hierarchy = nx.DiGraph()

hierarchy = process_hierarchy(graph, hierarchy)
root = get_root(hierarchy)

In [ ]:
nodes = []
for n, d in graph.nodes(data=True):
    el = from_nx_node(n, d)
    nodes.append(el)
    if not el.labels:
        el.labels.append(Label(text=el.id))

# add hierarchy nodes
for n, d in hierarchy.nodes(data=True):
    if n not in graph:
        el = from_nx_node(n, d)
        nodes.append(el)


context = Registry()
with context:
    el_map = build_shape_map(*nodes)

    # nest elements based on hierarchical edges
    for u, v in hierarchy.edges():
        parent = u if isinstance(u, Node) else el_map[u]
        child = v if isinstance(v, Node) else el_map[v]
        parent.children.append(child)

    edges = []
    for u, v, d in graph.edges(data=True):
        e_dict = {**d, "source": u, "target": v}
        edge = build_edge(e_dict, el_map)
        owner = get_owner(edge, hierarchy, el_map)
        owner.edges.append(edge)

In [ ]:
ipyelk.nx.Diagram(layout=dict(min_height="200px"))

In [ ]:
def a_flat_elk_json_example(graph: networkx.MultiDiGraph = None):
    graph = graph or networkx.readwrite.json_graph.node_link_graph(
        json.loads(pathlib.Path("flat_graph.json").read_text(encoding="utf-8"))
    )

    elk = ipyelk.ElkDiagram(layout=dict(min_height="200px"))
    xelk = ipyelk.nx.XELK(source=(graph, None))
    xelk.connect(elk)
    return elk, xelk

## Flat structure

A `networkx.MultiDigraph` can be used to create a flat graph.

> _TODO: There should be an option to specify if ports should be created or only connect
> edges between the nodes_

In [ ]:
def a_flat_elk_json_example(graph: networkx.MultiDiGraph = None):
    graph = graph or networkx.readwrite.json_graph.node_link_graph(
        json.loads(pathlib.Path("flat_graph.json").read_text(encoding="utf-8"))
    )

    elk = ipyelk.ElkDiagram(layout=dict(min_height="200px"))
    xelk = ipyelk.nx.XELK(source=(graph, None))
    xelk.connect(elk)
    return elk, xelk

In [ ]:
if __name__ == "__main__":
    flat, xflat = a_flat_elk_json_example()
    display(flat)

## Hierarchical Diagram with Ports

In [ ]:
def a_hierarchical_elk_example(
    tree: networkx.MultiDiGraph = None, ports: networkx.MultiDiGraph = None
):
    tree = tree or networkx.readwrite.json_graph.node_link_graph(
        json.loads(pathlib.Path("hier_tree.json").read_text(encoding="utf-8"))
    )
    ports = ports or networkx.readwrite.json_graph.node_link_graph(
        json.loads(pathlib.Path("hier_ports.json").read_text(encoding="utf-8"))
    )

    elk = ipyelk.ElkDiagram()
    xelk = ipyelk.nx.XELK(source=(ports, tree))
    xelk.connect(elk)

    return elk, xelk

In [ ]:
def a_collapsible_elk_example(elk_xelk=None):
    elk, xelk = elk_xelk or a_hierarchical_elk_example()
    ports, tree = xelk.source

    toggle = ipywidgets.Button(description="Toggle Collapsed")

    @toggle.on_click
    def toggle_node(widget):
        for element_id in elk.selected:
            if element_id in tree:
                for child in tree.neighbors(element_id):
                    state = tree.nodes[child].get("hidden", False)
                    tree.nodes[child]["hidden"] = not state
                xelk.refresh()

    box = ipywidgets.VBox(
        [
            ipywidgets.HBox(
                [ipywidgets.HTML("<h2>👇 click a group node then click 👉</h2>"), toggle]
            ),
            elk,
        ]
    )
    return box, elk, xelk

In [ ]:
if __name__ == "__main__":
    hier_box, hier_elk, hier_xelk = a_collapsible_elk_example()
    display(hier_box)

## 🦌 Learn More 📖

See the [other examples](./_index.ipynb).